# Assignment 2
## by Vinit Saah

#### Initialize base overlay and import libraries

In [1]:
import random
import threading
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pynq.lib.rgbled as rgbled 
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### blink LED. Based on time (t), counter value is calculated. blink rate determines rate of blinking LED

In [2]:
def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: time to blink the LED for a given task
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 3)
    '''
    x = 0
    t = round(t/d) #converting the time into counter value
    for i in range(t):
        if 0 <= n and n< 4:
            base.leds[n].toggle()
            time.sleep(d)
        elif n == 4:
            if x == 0:
                x = 2
            elif x == 2:
                x = 0
            rgbled.RGBLED(n).write(x) #Green color is 0x02
            time.sleep(d)
        else:
            print("Invalid LED IDs to blink")
            return
    if 0 <= n and n < 4:
        base.leds[n].off()
    if n == 4:
        rgbled.RGBLED(n).write(0)
    

### Test code to blink LED

In [ ]:
blink(1, 0.1, 0)
blink(2,0.02,4)
blink(2,0.02,7)

### In this approach, I maintain a state list. Philosophers are courteous to only eat when their neighbours are not eating. Default state of all Philosopher is STARVING initially.

In [10]:
EATING = 1
NAPPING = 2
STARVING = 3

def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            #print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining, data, mutex_state_mapping
    
    diner_state = philosopher_state[id_diner] 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    start_time = start_starving_time = time.time() #starving time 1, later to be reset after napping
    
    while halt == False:
        f_left_available = False
        f_right_available = False   
        #take a lock for shared list
        mutex_state_mapping.acquire(True)
        if philosopher_state[id_diner-1] != EATING and philosopher_state[(id_diner+1)%5]!=EATING:
            f_left_available = f_left.acquire(True) #It is safe to block as diners have seen neighours state.
            f_right_available = f_right.acquire(True)
        
        if f_right_available and f_left_available:
            diner_state = EATING
            philosopher_state[id_diner] = EATING
            end_starving_time = time.time()
            data['starving time']['Phil {}'.format(id_diner)] += end_starving_time-start_starving_time
        else:
            if f_right_available:
                f_right.release()
            if f_left_available:
                f_left.release()                 
        mutex_state_mapping.release()
            
        if diner_state == EATING:
            data['eat count']['Phil {}'.format(id_diner)] += 1
            start_eating_time = time.time()
            blink(2, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            mutex_state_mapping.acquire(True)
            philosopher_state[id_diner] = NAPPING
            mutex_state_mapping.release()
            end_eating_time = time.time()
            data['eating time']['Phil {}'.format(id_diner)] += end_eating_time - start_eating_time
    
        if diner_state == NAPPING:
            data['nap count']['Phil {}'.format(id_diner)] += 1
            start_napping_time = time.time()
            blink(1, 0.1,id_diner)
            end_napping_time = time.time()
            data['napping time']['Phil {}'.format(id_diner)] += end_napping_time-start_napping_time
            diner_state = STARVING
            start_starving_time = time.time()
            mutex_state_mapping.acquire(True)
            philosopher_state[id_diner] = STARVING
            mutex_state_mapping.release()
    
        if diner_state == STARVING:
            data['starve count']['Phil {}'.format(id_diner)] += 1
           

        button_lock.acquire(True)
        if stop_dining == True:
            halt = True
        button_lock.release()
    
    data['time sum']['Phil {}'.format(id_diner)] = data['eating time']['Phil {}'.format(id_diner)] + \
                                              data['napping time']['Phil {}'.format(id_diner)] + \
                                              data['starving time']['Phil {}'.format(id_diner)]
    data['run time']['Phil {}'.format(id_diner)] = int(time.time())-start_time
        
    
        
stop_dining = False

#Data metrics
cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)


fork_lock = list() #individual lock per philosopher
mutex_state_mapping = threading.Lock()
philosopher_state = list()
if __name__ == "__main__":    
    
    philosophers_threads = list() #List of thread
    button_lock = threading.Lock() #Lock for getting button status, as philosophers reads it and other thread write to it.
    btns = base.btns_gpio
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)
        philosopher_state.append(STARVING)

    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
   #Button press thread would exit first
    name = b_t.getName()
    b_t.join()
    print('Button Press Monitor {} joined'.format(name))
    
    #Philosophers would then exit depending upon their activity
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('Philosopher{} joined'.format(name))
    data = data.round(1)


Button Press Monitor Thread-34 joined
PhilosopherThread-35 joined
PhilosopherThread-36 joined
PhilosopherThread-37 joined
PhilosopherThread-38 joined
PhilosopherThread-39 joined


In [11]:
data

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,26.1,4.0,4.0,5209.0,8.2,10.9,4.1,23.3
Phil 1,27.0,4.0,4.0,5791.0,12.7,10.9,4.2,27.8
Phil 2,26.0,4.0,4.0,5575.0,10.1,10.9,4.3,25.3
Phil 3,28.0,4.0,4.0,6633.0,14.0,10.2,4.1,28.3
Phil 4,26.0,4.0,4.0,4759.0,10.6,11.0,4.2,25.9


In [12]:
EATING = 1
NAPPING = 2
STARVING = 3

def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            #print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining, data, mutex_state_mapping
    
    diner_state = philosopher_state[id_diner] 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    start_time = start_starving_time = time.time() #starving time 1, later to be reset after napping
    
    while halt == False:
        f_left_available = False
        f_right_available = False   
        #take a lock for shared list
        mutex_state_mapping.acquire(True)
        if philosopher_state[id_diner-1] != EATING and philosopher_state[(id_diner+1)%5]!=EATING:
            f_left_available = f_left.acquire(True) #It is safe to block as diners have seen neighours state.
            f_right_available = f_right.acquire(True)
        
        if f_right_available and f_left_available:
            diner_state = EATING
            philosopher_state[id_diner] = EATING
            end_starving_time = time.time()
            data['starving time']['Phil {}'.format(id_diner)] += end_starving_time-start_starving_time
        else:
            if f_right_available:
                f_right.release()
            if f_left_available:
                f_left.release()                 
        mutex_state_mapping.release()
            
        if diner_state == EATING:
            data['eat count']['Phil {}'.format(id_diner)] += 1
            start_eating_time = time.time()
            rand_eat = random.randint(5,10)
            blink(rand_eat, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            mutex_state_mapping.acquire(True)
            philosopher_state[id_diner] = NAPPING
            mutex_state_mapping.release()
            end_eating_time = time.time()
            data['eating time']['Phil {}'.format(id_diner)] += end_eating_time - start_eating_time
    
        if diner_state == NAPPING:
            data['nap count']['Phil {}'.format(id_diner)] += 1
            start_napping_time = time.time()
            rand_nap = random.randint(1,4)
            blink(rand_nap, 0.1,id_diner)
            end_napping_time = time.time()
            data['napping time']['Phil {}'.format(id_diner)] += end_napping_time-start_napping_time
            diner_state = STARVING
            start_starving_time = time.time()
            mutex_state_mapping.acquire(True)
            philosopher_state[id_diner] = STARVING
            mutex_state_mapping.release()
    
        if diner_state == STARVING:
            data['starve count']['Phil {}'.format(id_diner)] += 1
           

        button_lock.acquire(True)
        if stop_dining == True:
            halt = True
        button_lock.release()
    
    data['time sum']['Phil {}'.format(id_diner)] = data['eating time']['Phil {}'.format(id_diner)] + \
                                              data['napping time']['Phil {}'.format(id_diner)] + \
                                              data['starving time']['Phil {}'.format(id_diner)]
    data['run time']['Phil {}'.format(id_diner)] = int(time.time())-start_time
        
    
        
stop_dining = False

#Data metrics
cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)


fork_lock = list() #individual lock per philosopher
mutex_state_mapping = threading.Lock()
philosopher_state = list()
if __name__ == "__main__":    
    
    philosophers_threads = list() #List of thread
    button_lock = threading.Lock() #Lock for getting button status, as philosophers reads it and other thread write to it.
    btns = base.btns_gpio
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)
        philosopher_state.append(STARVING)

    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
   #Button press thread would exit first
    name = b_t.getName()
    b_t.join()
    print('Button Press Monitor {} joined'.format(name))
    
    #Philosophers would then exit depending upon their activity
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('Philosopher{} joined'.format(name))
    data = data.round(1)


Button Press Monitor Thread-40 joined
PhilosopherThread-41 joined
PhilosopherThread-42 joined
PhilosopherThread-43 joined
PhilosopherThread-44 joined
PhilosopherThread-45 joined


In [13]:
data

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,75.2,4.0,4.0,15399.0,30.7,34.9,10.3,75.9
Phil 1,74.2,3.0,3.0,15813.0,32.8,25.4,10.4,68.5
Phil 2,85.1,4.0,4.0,12361.0,27.3,44.7,13.3,85.2
Phil 3,74.1,3.0,3.0,16188.0,37.5,27.7,8.4,73.6
Phil 4,81.1,4.0,4.0,18913.0,42.2,29.5,10.4,82.1
